In [8]:
import tensorflow as tf, tensorflow
from tensorflow import keras
import torch
import cv2
import numpy
import os,sys,time
from ultralytics import YOLO
import numpy as np

In [1]:
depthModel = keras.models.load_model("models/depth estimation/modelforDepth.h5")

# Load your custom YOLO model for pothole detection
model = YOLO('DONT TOUCH THESE MODELS/pothole_id.pt')

# Function to preprocess the image for the depth model
def imgArray(img):
    img = tf.expand_dims(img, axis=-1)  # Add channel dimension (for grayscale image)
    img = tf.image.convert_image_dtype(img, tf.float32)
    img = tf.image.resize(img, [128, 128])
    return np.expand_dims(img, axis=0)  # Add batch dimension

# Load your pothole image
img = cv2.imread("Combined Data/train_images/p39.jpg")

# Perform object detection to identify potholes
result = model(img)

# Get the bounding box coordinates from the result
boxes = result[0].boxes.xyxy  # Get xyxy coordinates
scores = result[0].boxes.conf  # Get confidence scores

fontScale = 1
color = (0, 0, 255)
thickness = 2
font = cv2.FONT_HERSHEY_SIMPLEX

for i, box in enumerate(boxes):
    if scores[i] > 0:  # Check if confidence is greater than 0
        # Crop the detected pothole region
        x1, y1, x2, y2 = map(int, box)
        img1 = img[y1:y2, x1:x2]

        # Convert to grayscale
        img_ = cv2.cvtColor(img1, cv2.COLOR_BGR2GRAY)
        bluredImg = cv2.GaussianBlur(img_, (3, 3), 4400)
        _, thresh = cv2.threshold(bluredImg, 120, 255, cv2.THRESH_BINARY)
        
        # Load and preprocess the image for the depth model
        toFindDepth = imgArray(thresh)
        
        # Predict depth
        arr = depthModel.predict(toFindDepth)
        dep = arr[0][0][0][0]  # Assuming the depth model outputs depth directly
        
        # Annotate the original image with the depth value
        img = cv2.putText(img, f"{dep:.2f} cm", (x1 - 5, y1 - 5),
                          font, fontScale, color, thickness, cv2.LINE_AA)
        img = cv2.rectangle(img, (x1, y1), (x2, y2), (255, 0, 0), 2)

# Convert the image from BGR to RGB for plotting
img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

# Plot the final annotated image using Matplotlib
plt.figure(figsize=(10, 10))
plt.imshow(img_rgb)
plt.axis('off')  # Hide axes
plt.show()

NameError: name 'keras' is not defined